In [1]:
import pandas as pd
import personal_info
import requests
import threading
import time
from tqdm import tqdm
AK = '20222d44e5f1d4a9b879eedee30d9340'

In [2]:
df_poi_type = pd.read_excel('map_poicode.xlsx')
df_poi_type.head()

,序号,NEW_TYPE,大类,中类,小类,Big Category,Mid Category,Sub Category
0,1,10000,汽车服务,汽车服务相关,汽车服务相关,Auto Service,Automobile Service Related,Automobile Service Related
1,2,10100,汽车服务,加油站,加油站,Auto Service,Filling Station,Filling Station
2,3,10101,汽车服务,加油站,中国石化,Auto Service,Filling Station,Sinopec
3,4,10102,汽车服务,加油站,中国石油,Auto Service,Filling Station,PetroChina
4,5,10103,汽车服务,加油站,壳牌,Auto Service,Filling Station,Shell


In [3]:
df_poi_type.shape

(869, 8)

In [4]:
dic2name ={}
for i in range(df_poi_type.shape[0]):
    dic2name[df_poi_type['NEW_TYPE'].iloc[i]] = ','.join([df_poi_type['大类'].iloc[i],df_poi_type['中类'].iloc[i],df_poi_type['小类'].iloc[i]])

In [5]:
len(dic2name)

869

In [6]:
dic2name[int('10103')]

'汽车服务,加油站,壳牌'

In [7]:
#高德 获取地址的JSON   根据经度和纬度 顺序不要错
def get_request_json_by_gaode(lat,lng):
    location =','.join([str(lat),str(lng)])
    url = 'https://restapi.amap.com/v3/geocode/regeo?'
    params = {'key':AK, 'location':location, 'extensions':'all', 'radius':0}
    res = requests.get(url,params=params)
    time.sleep(0.01)
    return res.json()

In [8]:
lat = 106.282662
lng = 29.315744
lat1 = 106.500142
lng1 = 29.589695


In [9]:
def get_address_info_by_lat_lng(lat,lng):
    a = get_request_json_by_gaode(lat,lng)
    a = a['regeocode']
    formatted_address = a['formatted_address']
    city = a['addressComponent']['city']
    if city == []:
        city = ''
    province = a['addressComponent']['province']
    district = a['addressComponent']['district']
    township = a['addressComponent']['township']
    
    if a['aois'] != []:
        address_name = a['aois'][0]['name']
        # 以数字编号的形式 例如10010   10010|100012
        address_type_num = a['aois'][0]['type'].split('|')[0]
        address_type = dic2name[int(address_type_num)]
    else:
        address_name = a['pois'][0]['name']
        address_type = a['pois'][0]['type']
    return city,province,district,township,address_name,address_type

In [10]:

get_address_info_by_lat_lng(lat,lng)

('', '重庆市', '江津区', '德感街道', '敖家沟', '地名地址信息;普通地名;村庄级地名')

In [11]:
# def get_info(start,end):
#     for i in tqdm(range(start,end)):
#         lat = 106.282662
#         lng = 29.315744
#         get_address_info_by_lat_lng(lat,lng)

In [12]:
# start_time = time.time()
# import threading
# threads = []
# total = 35
# num_threading = 3
# each_threading_num = total // num_threading
# for i in range(num_threading):
#     if i == num_threading-1:
#         t = threading.Thread(target=get_info,args=(i*each_threading_num,total))
#     else:
#         t = threading.Thread(target=get_info,args=(i*each_threading_num,(i+1)*each_threading_num))
#     threads.append(t)
# for t in threads:
#     t.setDaemon(True)
#     t.start()
# print(time.time()-start_time)

In [13]:
df = pd.read_csv('a800_1000cars.csv')

c:\users\lianwei\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
a = '10010|00'
a.split('|')[0]

'10010'

In [15]:
classa = personal_info.Personal_analysis()

In [16]:
df = classa.process_data(df)

2019-05-11 22:38:37,241: INFO: 地址格式化完成.... 


In [17]:
#只计算周末
df_weekend = df[df['weekday'].isin([6,7])] 

In [18]:
df_weekend.shape

(309127, 22)

In [19]:
# 太多 删除一部分
user_id_in = list(set(df_weekend.user_id))[:-100]
df_weekend_a = df_weekend[df_weekend.user_id.isin(user_id_in)]

In [20]:
len(list(set(df_weekend.user_id))[:-100])

697

In [21]:
df_weekend_a.shape

(270488, 22)

In [22]:
# for i in tqdm(range(50,70)):
#     lat = df_weekend_a['start_gps_poi'].iloc[i].split(',')[0]
#     lng = df_weekend_a['start_gps_poi'].iloc[i].split(',')[1]
#     print(float(lat),float(lng))
#     get_address_info_by_lat_lng(float(lat),float(lng))


In [23]:
# df_weekend_a['city'] = '' 
# df_weekend_a['province'] = '' 
# df_weekend_a['district'] = '' 
# df_weekend_a['township'] = '' 
# df_weekend_a['address_name'] = '' 
# df_weekend_a['address_type'] = '' 
# #city,province,district,township,address_name,address_type

In [24]:
df_weekend_a.shape

(270488, 22)

In [25]:
df_weekend_a.columns

Index(['driving_history_id', 'car_id', 'user_id', 'start_address_name',
       'end_address_name', 'start_time', 'end_time', 'spend_time', 'speed_up',
       'speed_down', 'sharp_turn', 'over_speed', 'avrg_speed', 'mileage',
       'oil', 'fee', 'oil_price', 'create_at', 'start_gps_poi', 'end_gps_poi',
       'datetime', 'weekday'],
      dtype='object')

In [26]:
columns = ['user_id','car_id', 'start_time', 'end_time','start_gps_poi', 'end_gps_poi','datetime', 'weekday',
           'city', 'province', 'district', 'township', 'address_name', 'address_type']
df_weekend_b = pd.DataFrame(columns=columns)
df_weekend_b.head()

,user_id,car_id,start_time,end_time,start_gps_poi,end_gps_poi,datetime,weekday,city,province,district,township,address_name,address_type


In [27]:
columns = ['user_id']
df_weekend_c = pd.DataFrame(columns=columns)
df_weekend_c.head()

,user_id


In [28]:
df_weekend_c.loc[1] =['1']
df_weekend_c

,user_id
1,1


In [29]:
df_weekend_c.sort_index(axis=0)

,user_id
1,1


In [30]:
df_weekend_b.shape

(0, 14)

In [31]:
global count

In [32]:
count = 0
lock = threading.Lock()


In [34]:
def get_info_of_data(start,end):
    start_time_now = time.time()
    for i in range(start,end):   
        
        try:
            user_id = df_weekend_a['user_id'].iloc[i]
            print('%d----------%s' %(i,user_id))

            car_id = df_weekend_a['car_id'].iloc[i]
            start_time = df_weekend_a['start_time'].iloc[i]
            end_time = df_weekend_a['end_time'].iloc[i]
            start_gps_poi = df_weekend_a['start_gps_poi'].iloc[i]
            end_gps_poi = df_weekend_a['end_gps_poi'].iloc[i]
            datetime = df_weekend_a['datetime'].iloc[i]
            weekday = df_weekend_a['weekday'].iloc[i]
            #这里使用的是开始地址
            lat = df_weekend_a['start_gps_poi'].iloc[i].split(',')[0]
            lng = df_weekend_a['start_gps_poi'].iloc[i].split(',')[1]

            city, province, district, township,address_name, address_type = get_address_info_by_lat_lng(float(lat),float(lng))

        #     'city', 'province', 'district', 'township',
        #        'address_name', 'address_type'


            lock.acquire()
            df_weekend_b.loc[i] = [user_id,car_id, start_time, end_time,start_gps_poi, end_gps_poi,datetime, weekday,
                               city, province, district, township,address_name, address_type]
            print('写入第%d个信息：%s,%s,%s,%s,%s,%s,%s,%d,%s,%s,%s,%s,%s,%s,' %(i,user_id,car_id, start_time, end_time,start_gps_poi, end_gps_poi,datetime, weekday,
                               city, province, district, township,address_name, address_type))
            print('用时：%fs/n' %(time.time()-start_time_now))
        except Exception as e:
            print(e)
        finally:
            lock.release()

In [ ]:

start_time = time.time()

threads = []
#total = df_weekend_a.shape[0]
total = 60000
num_threading = 100
each_threading_num = total // num_threading
for i in range(num_threading):
    if i == num_threading-1:
        t = threading.Thread(target=get_info_of_data,args=(i*each_threading_num,total))
    else:
        t = threading.Thread(target=get_info_of_data,args=(i*each_threading_num,(i+1)*each_threading_num))
    threads.append(t)
for t in threads:
    t.setDaemon(True)
    t.start()
for t in threads:
    t.join()
print(time.time()-start_time)

0----------a0fdcaea-c01e-4b9f-b713-6f0eef9d4797600----------583f1e64-273f-4cca-ba7e-60fc186f52a61200----------899877187585310720


1800----------a0fdcaea-c01e-4b9f-b713-6f0eef9d47972400----------910758531529965568

3000----------9117805365763768323600----------9107750414303887364200----------1fa163d1-e693-49a2-ac6f-c5bbfaee451d
4800----------913603220960444416
5400----------8f3289e5-741d-4cf6-8c5f-68f688ef41d16000----------1fa163d1-e693-49a2-ac6f-c5bbfaee451d
6600----------913627767734534144写入第1800个信息：a0fdcaea-c01e-4b9f-b713-6f0eef9d4797,adb850fc-1b22-4f49-bbf1-e9f849ea344f,2017-09-16 19:55:39,2017-09-16 19:56:45.0,106.5746666,29.618166599999995,106.57366660000002,29.617500000000007,2017-09-16,6,,重庆市,江北区,寸滩街道,红沙坡,地名地址信息;普通地名;村庄级地名,7200----------859bf8db9e204410ada335f276c725477800----------25747e3e-fec3-4c3e-b8c9-809f7fcd6aac8400----------909694011814445056



9000----------9209088308653588489600----------aa6c88d59d1d48f1a877bc268fa337f5


10200----------83b3124b0c1840b48a5e257013feed9

写入第16200个信息：1abead5444c643d992a909f4ef25523d,917596654884880384,2017-11-26 17:32:55,2017-11-27 00:17:23.0,101.24349999999998,25.522833300000002,102.654,25.0373333,2017-11-26,7,楚雄彝族自治州,云南省,姚安县,栋川镇,冠君陶瓷,购物服务;家居建材市场;家居建材市场,
用时：2.381138s/n
16201----------9bdc1f7425e04f5ea340e5ed836c34ce
写入第16800个信息：83b3124b0c1840b48a5e257013feed9b,47346af0-f053-4b9b-834a-813f4dea22d2,2017-12-02 16:34:48,2017-12-02 16:38:45.0,116.0953333,34.14633330000001,116.0928333,34.14866660000001,2017-12-02,6,商丘市,河南省,夏邑县,郭店镇,郭店乡第一初级中学,科教文化服务;学校;中学,
用时：2.387124s/n
16801----------929681450746249216
写入第11400个信息：583f1e64-273f-4cca-ba7e-60fc186f52a6,898022495896993792,2017-11-04 13:29:43,2017-11-04 13:40:14.0,117.02749999999997,39.76483329999999,116.99950000000001,39.757000000000005,2017-11-04,6,廊坊市,河北省,香河县,淑阳镇,晨汐,商务住宅,住宅区,住宅小区,
用时：2.654152s/n
11401----------1abead5444c643d992a909f4ef25523d
写入第7200个信息：859bf8db9e204410ada335f276c72547,919849294637301760,2017-10-21 14:08:34,2017-10-21 14:11:01.0,99.9028333,26.5193333,99.90316

23401----------22ada4bde09b414599aff1ac1b88453b写入第27000个信息：22ada4bde09b414599aff1ac1b88453b,927716608921567232,2018-01-20 10:22:10,2018-01-20 11:02:35.0,101.4268333,26.502166599999995,101.4256666,26.5073333,2018-01-20,6,丽江市,云南省,华坪县,石龙坝镇,干沟湾,地名地址信息;普通地名;村庄级地名,

用时：2.441141s/n
27001----------52f1c5c0-f4dd-4956-87c9-b1c5fac81556写入第25200个信息：913603220960444416,913602721561444352,2018-01-13 12:08:36,2018-01-13 12:16:25.0,102.75483329999997,24.9556666,102.75233329999998,24.959166600000003,2018-01-13,6,昆明市,云南省,官渡区,官渡街道,尚义村,地名地址信息;普通地名;村庄级地名,

用时：2.523146s/n
25201----------934962453949710336写入第24000个信息：c1593401-2837-48e7-9ee1-7c3fc4ee8b5e,05b0c49e-a8b7-4048-967d-440834785db5,2018-01-06 18:20:13,2018-01-06 18:49:53.0,116.76116660000002,23.489666600000007,116.7608333,23.489666600000007,2018-01-06,6,汕头市,广东省,澄海区,广益街道,公共厕所,公共设施;公共厕所;公共厕所,

用时：2.583150s/n
24001----------914027069401202688写入第29400个信息：64c107be-9bb9-4e6d-9126-05e28c896b70,922296366250590208,2018-01-27 19:19:56,2018-01-27 19:40:36.0,112.


用时：2.333132s/n
39601----------f8c43e0148b043769a82834272cf5d61写入第41400个信息：e46a0876dff843e2bb50897eea95d24b,925979324920627200,2018-02-18 16:11:44,2018-02-18 16:26:02.0,124.05483329999998,42.531666599999994,124.06033330000002,42.56716660000001,2018-02-18,7,铁岭市,辽宁省,开原市,新城街道,溪桂庄园,商务住宅;商务住宅相关;商务住宅相关,

用时：2.290131s/n
41401----------918373019594784768写入第1201个信息：272da7b29589460d94bc9e8e25a67673,daba747b-cce0-4cc0-a9ae-afccd20cf5ff,2017-09-03 16:11:06,2017-09-03 16:14:27.0,117.52516659999998,34.88666660000001,117.5311666,34.89099999999999,2017-09-03,7,枣庄市,山东省,市中区,齐村镇,S83枣庄连接线高速出口(东向),地名地址信息;交通地名;高速路出口,

用时：4.241235s/n
1202----------272da7b29589460d94bc9e8e25a67673
写入第40800个信息：fdce5b77-9d0a-476b-9afe-1b3f14d1a400,958884291951984640,2018-02-18 12:25:59,2018-02-18 12:30:11.0,114.13783330000001,41.6551666,114.11583330000002,41.6418333,2018-02-18,7,乌兰察布市,内蒙古自治区,商都县,大黑沙土镇,中国邮政储蓄银行(大黑沙土邮局),金融保险服务;银行;中国邮政储蓄银行,
用时：2.357135s/n
40801----------6cae4bbcb2184f69a9d5077fee936d14写入第43200个信息：ed4f3af8-a705-469

用时：1.912110s/n
59401----------d147fa193e5c424bbe81c20631a8644a写入第57000个信息：6902986b870842978acbc2d189bc0796,800142b8-6f99-4d85-bcbd-3d30ffcfd91d,2018-03-18 10:16:54,2018-03-18 10:37:06.0,111.67783329999997,27.691999999999993,111.64933330000002,27.691999999999993,2018-03-18,7,娄底市,湖南省,涟源市,蓝田街道,涟源第一小学,科教文化服务,学校,小学,

用时：2.055118s/n
57001----------3e303bfe-f790-4d15-a5d3-cc6012ab13c1写入第55200个信息：c74390e146ef488dac05192d29020917,952744752170663936,2018-03-17 10:59:15,2018-03-17 11:05:32.0,118.8701666,31.944999999999993,118.87400000000002,31.938999999999993,2018-03-17,6,南京市,江苏省,江宁区,淳化街道,中共江宁区委员会党校,科教文化服务,学校,学校,

用时：2.176126s/n
55201----------40db3390-f1d4-469a-9900-1287751fce28
写入第58800个信息：64c107be-9bb9-4e6d-9126-05e28c896b70,922296366250590208,2018-03-24 09:02:12,2018-03-24 09:30:33.0,112.48349999999999,33.012333299999995,112.48349999999999,33.0121666,2018-03-24,6,南阳市,河南省,卧龙区,百里奚街道,天一驾校训练场,科教文化服务;驾校;驾校,
用时：1.989116s/n
58801----------bb5b4279-de61-4e18-8350-8b6736dcdaf3
写入第2401个信息：9096940118144

...

写入第49201个信息：fdce5b77-9d0a-476b-9afe-1b3f14d1a400,958884291951984640,2018-03-04 09:41:27,2018-03-04 09:47:16.0,114.83966659999999,40.77466659999999,114.85666659999998,40.769499999999994,2018-03-04,7,张家口市,河北省,桥西区,沈家屯镇,张家口春雷医院,医疗保健服务,综合医院,综合医院,
用时：4.834280s/n
49202----------961115723248173056
写入第46801个信息：961150896123674624,a30e1609-764e-4477-871b-fb8ecef7097e,2018-03-03 08:30:17,2018-03-03 09:18:14.0,113.25,26.751000000000005,113.26516659999999,26.715666600000006,2018-03-03,6,郴州市,湖南省,安仁县,永乐江镇,罗丫坊,地名地址信息;普通地名;村庄级地名,
用时：4.967287s/n
46802----------a11c77cb-819d-4c83-92c4-2698de44a944
写入第52801个信息：962557427935346688,961777613485899776,2018-03-11 07:01:17,2018-03-11 07:26:45.0,116.38850000000002,31.2608333,116.41533329999999,31.27949989999999,2018-03-11,7,六安市,安徽省,霍山县,单龙寺镇,莲花地生态茶园,体育休闲服务;休闲场所;休闲场所,
用时：4.710270s/n
52802----------936429447064584192
写入第53401个信息：50d64f2d-d93f-4798-9c15-5eda4c1f937a,01179eb5-2ee4-417b-9218-aef7b8ec004a,2018-03-11 11:54:37,2018-03-11 12:00:44.0,116.72166659999999,36.2

写入第10202个信息：918314718681104384,918314039421960192,2017-10-29 10:58:02,2017-10-29 11:46:30.0,113.32633329999999,23.2711666,113.3303333,23.267333300000004,2017-10-29,7,广州市,广东省,白云区,太和镇,铭艺美容美发美甲家具展示厅,购物服务;家居建材市场;家具城,
用时：7.641439s/n
10203----------910033494313271296
写入第12602个信息：911423423484264448,911423059284459520,2017-11-05 19:01:31,2017-11-05 19:01:49.0,118.8941666,34.32033329999999,118.894,34.32033329999999,2017-11-05,7,宿迁市,江苏省,沭阳县,青伊湖镇,滥洪村,地名地址信息;普通地名;村庄级地名,
用时：7.578437s/n
12603----------909694011814445056
写入第15602个信息：95a07607625a4dbdadeebf6f88b4f8a3,901971892460584960,2017-11-25 18:29:44,2017-11-25 18:56:21.0,110.12316659999999,36.28049999999999,110.11950000000002,36.282333300000005,2017-11-25,6,延安市,陕西省,宜川县,云岩镇,云岩镇人民政府,政府机构及社会团体;政府机关;乡镇级政府及事业单位,
用时：7.475430s/n
15603----------3e303bfe-f790-4d15-a5d3-cc6012ab13c1
写入第7802个信息：909650316759138304,907834233567838208,2017-10-22 07:54:47,2017-10-22 07:59:50.0,117.64583329999999,34.6143333,117.64616660000002,34.6153333,2017-10-22,7,枣庄市,山东省,台儿庄区

28202----------2eff5b2a-bbb9-4a4e-ab36-efc502945103写入第25802个信息：6a27af95-a4ae-40ef-92f4-0c1d1fa71e89,8139afa5-1d14-49b8-a8ae-4847b50350dc,2018-01-14 06:30:04,2018-01-14 07:09:10.0,118.6504999,35.588833300000005,118.6308333,35.57733329999999,2018-01-14,7,临沂市,山东省,沂南县,湖头镇,临沂市公安局交通警察支队湖头中队,政府机构及社会团体;交通车辆管理;交通管理机构,

用时：7.912454s/n
25803----------6906798a4f0745ffaf0cdfc6313fe952
写入第24602个信息：917958594366799872,917958402758410240,2018-01-07 14:13:11,2018-01-07 14:18:48.0,113.11333330000002,22.986500000000007,113.1093333,22.9826666,2018-01-07,7,佛山市,广东省,禅城区,石湾镇街道,黎冲朝阳新村,商务住宅,住宅区,住宅小区,
用时：7.965459s/n
24603----------a92f611b-1d31-49b2-b61c-cfa32e85e5ed
写入第13202个信息：null,e6d2f1c9-5fdb-48ae-a5cb-d3efdb996d64,2017-11-11 18:15:39,2017-11-11 18:27:32.0,105.3871666,30.878166599999986,105.3833333,30.888833299999988,2017-11-11,6,遂宁市,四川省,射洪县,平安街道,璧合苑,商务住宅,住宅区,住宅小区,
用时：8.578494s/n
13203----------912153828848041984
写入第32402个信息：034ecc1d57a141f39804b009fb6af174,47923323-0499-4e80-8c5e-ed83469b69d0,2018-02-04 16:

写入第40802个信息：913627767734534144,913626741010857984,2018-02-18 12:26:12,2018-02-18 13:52:26.0,110.85199990000001,27.023333300000004,111.43,27.238500000000002,2018-02-18,7,邵阳市,湖南省,洞口县,黄桥镇,湘运黄桥汽车站,交通设施服务,长途汽车站,长途汽车站,
用时：7.981459s/n
40803----------ed9741dddf5c46439617068c368b5fd3
写入第43202个信息：962220027820376064,b2cf8e99-f8d5-4159-8b15-fbbf81c304c9,2018-02-24 13:37:59,2018-02-24 13:55:18.0,105.77833329999999,31.474666600000006,105.75066659999999,31.486666600000007,2018-02-24,6,南充市,四川省,南部县,升钟镇,陈家沟,地名地址信息;普通地名;村庄级地名,
用时：7.906457s/n
43203----------453a22ec-d79b-4a46-8fa9-65706218a02a
写入第39002个信息：453a22ec-d79b-4a46-8fa9-65706218a02a,904518887981187072,2018-02-17 15:08:01,2018-02-17 15:10:43.0,115.22050000000002,34.101,115.22216659999998,34.10266659999999,2018-02-17,6,商丘市,河南省,柘城县,岗王镇,岗王茗城家具店,购物服务;家居建材市场;家居建材市场,
用时：8.133468s/n
39003----------953093699644948480
写入第45002个信息：a626c332-af14-4b15-80b0-452eed6c69a2,958962220727795712,2018-02-25 11:15:29,2018-02-25 11:39:52.0,107.31150000000002,33.137,107.

写入第603个信息：a0fdcaea-c01e-4b9f-b713-6f0eef9d4797,adb850fc-1b22-4f49-bbf1-e9f849ea344f,2017-08-19 15:29:17,2017-08-19 15:33:14.0,106.4893333,29.605,106.4921666,29.603,2017-08-19,6,,重庆市,江北区,石马河街道,江北石子山体育公园,风景名胜,公园广场,公园,
用时：10.952631s/n
604----------40db3390-f1d4-469a-9900-1287751fce28
写入第56402个信息：286e8094-539d-40f1-b642-6a12b1654ba7,272e37d0-0a69-4c78-bb11-8811fb15d8c4,2018-03-17 21:49:33,2018-03-17 22:01:12.0,133.5265,46.84449989999999,133.5241666,46.84316659999999,2018-03-17,6,双鸭山市,黑龙江省,饶河县,石场林场,博祥医药(石场分店),医疗保健服务;医药保健销售店;药房,
用时：8.248476s/n
56403----------f5440c24-9f5f-43eb-bb2c-0a14128f09ed
写入第6003个信息：8f3289e5-741d-4cf6-8c5f-68f688ef41d1,916502013720657920,2017-10-14 20:11:28,2017-10-14 20:22:51.0,113.54183330000001,34.493166599999995,113.56466660000001,34.471166600000004,2017-10-14,6,郑州市,河南省,新密市,刘寨镇,百福家宴,餐饮服务;中餐厅;中餐厅,
用时：10.703601s/n
6004----------71445bdd22bd41589c7cc93534282720
写入第4803个信息：c1593401-2837-48e7-9ee1-7c3fc4ee8b5e,05b0c49e-a8b7-4048-967d-440834785db5,2017-10-08 02:41:18,201

In [ ]:
# t = tqdm(range(100))
# for i in t:
#     time.sleep(0.01)
# t.close()
start_time = time.time()
print('用时：%fs/n' %(time.time()-start_time))

In [ ]:
# df_weekend_a.loc[288219,'city'] = 'b'
len(set(df_weekend_b['start_gps_poi']))

In [35]:
df_weekend_b.shape

(6000, 14)

In [ ]:
g = df_weekend_b.sort_index(axis=0)

In [ ]:
set([11,2,3,1],[1,2,4])

In [ ]:
loss_index = []
for i in range(3000):
    if i not in g.index:
        loss_index.append(i)

In [ ]:
len(loss_index)

In [ ]:
# df_weekend_a.head()

In [ ]:
# import time
# from tqdm import tqdm

In [ ]:
# for i in tqdm(range(100)):
#     time.sleep(0.1)